## Imports

In [18]:
import torchvision.datasets as ds
from sklearn import datasets
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import pickle

## Dense Layer

In [19]:
class DenseLayer:
    def __init__(self, inputDimensions, outputDimensions):
        self.input = None
        self.output = None
        self.weights = np.random.randn(outputDimensions, inputDimensions)
        self.biases = np.zeros((outputDimensions, 1))

    def forwardPass(self, input):
        self.input = input
        self.output = np.dot(self.weights, input) + self.biases
        return self.output
    
    def backwardPass(self, dL_dout, alpha):
        dw = np.dot(dL_dout, self.input.T)
        db = np.sum(dL_dout, axis=1, keepdims=True)

        di = np.dot(self.weights.T, dL_dout)

        self.weights -= alpha * dw
        self.biases -= alpha * db

        return di

### RELU

In [20]:
class RELU:
    ## matrix that passes through relu is (n[l], m)
    def forwardPass(self, x):
        self.input = x
        return np.maximum(0, x)
    
    def backwardPass(self, x):
        return x * (self.input > 0)

### Softmax

In [21]:
class Softmax:
    def calcSoftmax(self, Y):
        #calculate from an 1D array
        exps = np.exp(Y - np.max(Y, axis=0, keepdims=True))
        return exps / np.sum(exps, axis=0, keepdims=True)
    
    def forwardPass(self, Y):
        return self.calcSoftmax(Y)
    
    def backwardPass(self, Y):
        return Y

#### Loss calc

In [22]:
def crossEntropyLoss(y, y_hat):
    return -np.sum(y * np.log(y_hat + 1e-9)) / y.shape[1]

## Loaders

In [23]:
class Loaders:
    def train_data(self, epochs, learnRate, input_dimensions, output_dimensions, train_loader, dense1, activation1, dense2):
        #initialize layers
        for epoch in range(epochs):
            total_loss = 0
            for images, labels in train_loader:
                images = images.view(images.size(0), -1).numpy()
                labels_onehot = np.eye(output_dimensions)[labels.numpy()]

                #forward pass
                dense1_output = dense1.forwardPass(images.T)
                activation1_output = activation1.forwardPass(dense1_output)
                dense2_output = dense2.forwardPass(activation1_output)
                y_pred = Softmax().calcSoftmax(dense2_output)


                loss = crossEntropyLoss(labels_onehot, y_pred.T)
                total_loss += loss

                #backward pass
                loss_gradient = y_pred.T - labels_onehot
                dh = dense2.backwardPass(loss_gradient.T, learnRate)
                do1 = activation1.backwardPass(dh)
                dense1.backwardPass(do1, learnRate)

            print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

    def test_data(self, test_loader, dense1, activation1, dense2):
        
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.view(images.size(0), -1).numpy()
            labels = labels.numpy()

            #write like train
            
            dense1_output = dense1.forwardPass(images.T)
            activation1_output = activation1.forwardPass(dense1_output)
            dense2_output = dense2.forwardPass(activation1_output)
            y_pred = Softmax().calcSoftmax(dense2_output)

            predictions = np.argmax(y_pred, axis=0)
            correct += np.sum(predictions == labels)
            total += labels.shape[0]


        print(f"Accuracy: {correct/total}")

    def savetoPickle(self, dense1, dense2, filename):
        weghts_and_biases = {
            "dense1_weights": dense1.weights,
            "dense1_biases": dense1.biases,
            "dense2_weights": dense2.weights,
            "dense2_biases": dense2.biases
        }

        with open(filename, 'wb') as file:
            pickle.dump(weghts_and_biases, file)

        print("Saved to pickle file")

    def loadfromPickle(self, filename):
        with open(filename, 'rb') as file:
            weights_and_biases = pickle.load(file)

        dense1 = DenseLayer(28*28, 128)
        dense1.weights = weights_and_biases["dense1_weights"]
        dense1.biases = weights_and_biases["dense1_biases"]

        dense2 = DenseLayer(128, 10)
        dense2.weights = weights_and_biases["dense2_weights"]
        dense2.biases = weights_and_biases["dense2_biases"]

        return dense1, dense2


## Get train and test

In [24]:
transform = transforms.ToTensor()

train_dataset = datasets.FashionMNIST(root='./FMNIST', train=True, download=True, transform=transform)

test_dataset = datasets.FashionMNIST(root='./FMNIST', train=False, download=True, transform=transform)

## Initialization

In [25]:
loaders = Loaders()

input_dimensions = 28*28
output_dimensions = 10
numOfNeurons = 20

epochs = 5
learnRate = 0.1

dense1 = DenseLayer(input_dimensions, numOfNeurons)
activation1 = RELU()
dense2 = DenseLayer(numOfNeurons, output_dimensions)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

## Train

In [26]:
loaders.train_data(epochs, learnRate, input_dimensions, output_dimensions, train_loader, dense1, activation1, dense2)

Epoch 1, Loss: 24.50027656219117
Epoch 2, Loss: 23.509830998466914
Epoch 3, Loss: 23.503460824627844
Epoch 4, Loss: 23.528032840634907
Epoch 5, Loss: 23.532574532262622


## Test

In [27]:
loaders.test_data(test_loader, dense1, activation1, dense2)

Accuracy: 0.1


### Save to pickle

In [ ]:
filename = "1905019.pkl"
loaders.savetoPickle(dense1, dense2, filename)

### Load pickle

In [ ]:
loaders.loadfromPickle(filename)